In [ ]:
import os
import pandas as pd
import numpy as np
from IPython.core.display import display, HTML
import pymongo
from pymongo import MongoClient
#print ('Mongo version' pymongo.__version__)
client = MongoClient('localhost', 27017)
## Identifier la base de données 
db = client.test
## Assigner la collection correspondante à cette base de données
collection = db.restaurants

In [2]:
#mongo "db.version()" ANY

In [3]:
print(client.list_database_names())

['admin', 'config', 'local', 'patient_data', 'test']


In [6]:
test = client['test']

In [7]:
print(test.list_collection_names())

['restaurants', 'neighborhoods']


In [8]:
collection.find_one()

{'_id': ObjectId('55cba2476c522cafdb053ade'),
 'location': {'coordinates': [-73.961704, 40.662942], 'type': 'Point'},
 'name': "Wendy'S"}

In [9]:
from pymongo import MongoClient,ASCENDING,DESCENDING
client = MongoClient("mongodb://localhost:27017")
dbnames = [dbname for dbname in client.database_names() ]
print("Available databases: <" + str(dbnames))

Available databases: <['admin', 'config', 'local', 'patient_data', 'test']


/Users/garbamoussa/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: database_names is deprecated. Use list_database_names instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:
cursor = collection.find()
print()
print("Number of documents in restaurant collection: " + str( cursor.count() ))
N=4
print("1st {} documents:".format(N))
for document in cursor.limit(N):
    print(document)


Number of documents in restaurant collection: 25359
1st 4 documents:
{'_id': ObjectId('55cba2476c522cafdb053ade'), 'location': {'coordinates': [-73.961704, 40.662942], 'type': 'Point'}, 'name': "Wendy'S"}
{'_id': ObjectId('55cba2476c522cafdb053add'), 'location': {'coordinates': [-73.856077, 40.848447], 'type': 'Point'}, 'name': 'Morris Park Bake Shop'}
{'_id': ObjectId('55cba2476c522cafdb053ae0'), 'location': {'coordinates': [-73.8601152, 40.7311739], 'type': 'Point'}, 'name': 'Tov Kosher Kitchen'}
{'_id': ObjectId('55cba2476c522cafdb053ae1'), 'location': {'coordinates': [-73.8803827, 40.7643124], 'type': 'Point'}, 'name': 'Brunos On The Boulevard'}


/Users/garbamoussa/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:
cursor = collection.find().sort([ ('date', ASCENDING) ])
print()
print("Number of documents in restaurant collection: " + str( cursor.count() ))
N=4
print("1st {} documents: ORDERED by ASCENDING date".format(N))
for document in cursor.limit(N):
    print(document)


Number of documents in restaurant collection: 25359
1st 4 documents: ORDERED by ASCENDING date
{'_id': ObjectId('55cba2476c522cafdb053ade'), 'location': {'coordinates': [-73.961704, 40.662942], 'type': 'Point'}, 'name': "Wendy'S"}
{'_id': ObjectId('55cba2476c522cafdb053add'), 'location': {'coordinates': [-73.856077, 40.848447], 'type': 'Point'}, 'name': 'Morris Park Bake Shop'}
{'_id': ObjectId('55cba2476c522cafdb053ae0'), 'location': {'coordinates': [-73.8601152, 40.7311739], 'type': 'Point'}, 'name': 'Tov Kosher Kitchen'}
{'_id': ObjectId('55cba2476c522cafdb053ae1'), 'location': {'coordinates': [-73.8803827, 40.7643124], 'type': 'Point'}, 'name': 'Brunos On The Boulevard'}


/Users/garbamoussa/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [123]:
#collection.drop()
# /Users/garbamoussa/Desktop/restaurants.json
# /Users/garbamoussa/Desktop/neighborhoods.json
#os.system('mongoimport -d test -c restautants restaurants.json')

In [174]:
# Afficher toutes les données de la base de données
## cursor est un itérator permettant de créer une liste, après la création de cette liste l'itérator est épuisée
#cursor = collection.find()
cursor = collection.find().sort('zipcode',pymongo.ASCENDING)
#for doc in cursor:
 #   print (doc)

In [205]:
cursor = list(cursor)
#cursor

In [176]:
#cursorResult = collection.aggregate(cursor) # returns a cursor

In [12]:
#cursorResult = collection.aggregate(cursor) # returns a cursor
#cursor = pd.DataFrame(list(cursorResult)) 

data_frame_cursor = pd.DataFrame(cursor) 
data_frame_cursor = data_frame_cursor.set_index("_id")

KeyError: "None of ['_id'] are in the columns"

In [13]:
data_frame_cursor.head()

""


In [197]:
pipeline = [
        {"$group": {"_id":"$_id",
             "AvgScore":{"$avg":"$Score"},
             "Count":{"$sum":1},
        }},
        {"$sort":{"Count":-1,"AvgScore":1}}
]
aggResult = collection.aggregate(pipeline) # returns a cursor
df1 = pd.DataFrame(list(aggResult)) # use list to turn the cursor to an array of documents
#df1 = df1.set_index("_id")
df1.head()

,AvgScore,Count,_id
0,None,1,55cba2476c522cafdb054a72
1,None,1,55cba2476c522cafdb053d8a
2,None,1,55cba2476c522cafdb058025
3,None,1,55cba2476c522cafdb058fae
4,None,1,55cba2476c522cafdb0568ff


In [198]:
pipeline = [
        {"$match": {"name":"Morris Park Bake Shop"}},
]
aggResult = collection.aggregate(pipeline)
df2 = pd.DataFrame(list(aggResult))
df2.head()

,_id,location,name
0,55cba2476c522cafdb053add,"{'coordinates': [-73.856077, 40.848447], 'type...",Morris Park Bake Shop


In [200]:
pipeline = [
        {"$match": {"name":"Wilken'S Fine Food"}},
]
aggResult = collection.aggregate(pipeline)
df2 = pd.DataFrame(list(aggResult))
df2.head()

,_id,location,name
0,55cba2476c522cafdb053ae3,"{'coordinates': [-73.9068506, 40.6199034], 'ty...",Wilken'S Fine Food


In [201]:
df2.dtypes

_id         object
location    object
name        object
dtype: object

In [202]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 3 columns):
_id         1 non-null object
location    1 non-null object
name        1 non-null object
dtypes: object(3)
memory usage: 104.0+ bytes


In [203]:
type(df2)

pandas.core.frame.DataFrame

In [204]:
pipeline = [
        {"$match": {"cuisine":"Chinese"}},
]
aggResult = collection.aggregate(pipeline)
df3 = pd.DataFrame(list(aggResult))
df3.head()

""


In [195]:
import folium
#print 'Folium version', folium.__version__

world_map = folium.Map(location=[35, 100], 
                    zoom_start=4)
#for i in range(len(df2)):
    #world_map.simple_marker(location=df2.Location[i].split(','), popup=df2.Name[i]+', borough:'+str(df2.borough[i]))
    
world_map

In [194]:
import folium
#print 'Folium version', folium.__version__

world_map = folium.Map(location=[35, 100], 
                    zoom_start=4)
folium.Marker([35, 100], popup='London Bridge').add_to(world_map)
world_map


In [188]:
from pymongo import MongoClient

client = MongoClient('mongodb://localhost:27017/')

with client:
    
    db = client.test
    print(db.collection_names())

['restaurants']


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: collection_names is deprecated. Use list_collection_names instead.
  


In [189]:
from pymongo import MongoClient
from pprint import pprint

client = MongoClient('mongodb://localhost:27017/')

with client:
    
    db = client.test

    status = db.command("serverStatus")
    pprint(status)

{'asserts': {'msg': 0, 'regular': 0, 'rollovers': 0, 'user': 11, 'warning': 0},
 'connections': {'active': 1,
                 'available': 186,
                 'current': 18,
                 'totalCreated': 349},
 'extra_info': {'note': 'fields vary by platform', 'page_faults': 130},
 'flowControl': {'enabled': True,
                 'isLagged': False,
                 'isLaggedCount': 0,
                 'isLaggedTimeMicros': 0,
                 'locksPerOp': 0.0,
                 'sustainerRate': 0,
                 'targetRateLimit': 1000000000,
                 'timeAcquiringMicros': 0},
 'freeMonitoring': {'state': 'undecided'},
 'globalLock': {'activeClients': {'readers': 0, 'total': 0, 'writers': 0},
                'currentQueue': {'readers': 0, 'total': 0, 'writers': 0},
                'totalTime': 9902814000},
 'host': 'MBP-de-GARBA',
 'localTime': datetime.datetime(2019, 9, 1, 23, 47, 54, 762000),
 'locks': {'Collection': {'acquireCount': {'R': 1,
                       

In [190]:
from pymongo import MongoClient
from pprint import pprint

client = MongoClient('mongodb://localhost:27017/')

with client:
    
    db = client.test
    print(db.collection_names())

    status = db.command("dbstats")
    pprint(status)

['restaurants']
{'avgObjSize': 252.27134695155357,
 'collections': 1,
 'dataSize': 6446542.0,
 'db': 'test',
 'fsTotalSize': 250685575168.0,
 'fsUsedSize': 136135819264.0,
 'indexSize': 483328.0,
 'indexes': 1,
 'numExtents': 0,
 'objects': 25554,
 'ok': 1.0,
 'scaleFactor': 1.0,
 'storageSize': 4587520.0,
 'views': 0}


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: collection_names is deprecated. Use list_collection_names instead.
  if __name__ == '__main__':


In [191]:
#cars = db.test.find()

In [192]:
cars

NameError: name 'cars' is not defined

In [193]:
df2.head()

""


In [44]:
df2.tail()

,_id,address,borough,cuisine,grades,name,restaurant_id
2333,5c682d084f8f574fb5e15045,"{'building': '3505', 'coord': {'type': 'Point'...",Bronx,Other,[],,50018891
2334,5c682d084f8f574fb5e15049,"{'building': '556', 'coord': {'type': 'Point',...",Bronx,Other,[],,50018902
2335,5c682d084f8f574fb5e15052,"{'building': '600', 'coord': {'type': 'Point',...",Bronx,Other,[],,50018868
2336,5c682d084f8f574fb5e15055,"{'building': '689', 'coord': {'type': 'Point',...",Bronx,Other,[],187Th St Pizza,50018939
2337,5c682d084f8f574fb5e15056,"{'building': '2009', 'coord': {'type': 'Point'...",Bronx,Other,[],Subway,50018940


In [48]:
df3.tail()

,_id,address,borough,cuisine,grades,name,restaurant_id
2413,5c682d084f8f574fb5e14f79,"{'building': '5723', 'coord': {'type': 'Point'...",Brooklyn,Chinese,"[{'date': 2015-01-20 00:00:00, 'grade': 'Not Y...",Uncle Wang Chinese Food,50018450
2414,5c682d084f8f574fb5e14f84,"{'building': '108', 'coord': {'type': 'Point',...",Manhattan,Chinese,"[{'date': 2015-01-20 00:00:00, 'grade': 'Not Y...",Savour Sichuan,50018468
2415,5c682d084f8f574fb5e14f97,"{'building': '65', 'coord': {'type': 'Point', ...",Manhattan,Chinese,"[{'date': 2015-01-20 00:00:00, 'grade': 'Not Y...",Bubble Tea,50018503
2416,5c682d084f8f574fb5e14fa9,"{'building': '7706', 'coord': {'type': 'Point'...",Brooklyn,Chinese,"[{'date': 2015-01-20 00:00:00, 'grade': 'Not Y...",Orchid Dynasty Restaurant,50018552
2417,5c682d084f8f574fb5e14fd5,"{'building': '6022', 'coord': {'type': 'Point'...",Brooklyn,Chinese,"[{'date': 2015-01-20 00:00:00, 'grade': 'Not Y...",New Star Seafood Restaurant,50018622


In [49]:
df2.groupby("borough").sum()

,cuisine,grades,name,restaurant_id
borough,,,,
Bronx,"BakeryAmerican Ice Cream, Gelato, Yogurt, Ices...","[{'date': 2014-03-03 00:00:00, 'grade': 'A', '...",Morris Park Bake ShopWild AsiaCarvel Ice Cream...,3007544540357217403630934036328940364296403643...


In [50]:
df3.groupby("borough").sum()

,cuisine,grades,name,restaurant_id
borough,,,,
Bronx,ChineseChineseChineseChineseChineseChineseChin...,"[{'date': 2013-12-30 00:00:00, 'grade': 'A', '...",Happy GardenHappy GardenChina Wok IiDragon Cit...,4036328940364296405103284052920340551996405522...
Brooklyn,ChineseChineseChineseChineseChineseChineseChin...,"[{'date': 2014-09-16 00:00:00, 'grade': 'B', '...",May May KitchenGolden PavillionLee'S Villa Chi...,4035842940363920403883864039065340391528403931...
Manhattan,ChineseChineseChineseChineseChineseChineseChin...,"[{'date': 2014-07-28 00:00:00, 'grade': 'B', '...",Hop Kee RestaurantShun Lee Palace RestaurantSh...,4036594240366716403739374037612140382877403862...
Missing,ChineseChineseChineseChineseChineseChinese,"[{'date': 2014-05-30 00:00:00, 'grade': 'A', '...",Unique BakeryNew Win Hing IncLoong Xin Asian C...,500034925000371950004484500046845000534150006223
Queens,ChineseChineseChineseChineseChineseChineseChin...,"[{'date': 2014-04-21 00:00:00, 'grade': 'A', '...",Ho Mei RestaurantKing Yum RestaurantJardin De ...,4036243240365592403665434038926640391498403927...
Staten Island,ChineseChineseChineseChineseChineseChineseChin...,"[{'date': 2014-12-11 00:00:00, 'grade': 'A', '...",Chen'S ChineseIsland TasteMaster WokHappy Fort...,4037283140394383403943944040109440536227405534...


In [51]:
df3.groupby(["borough", "name"]).sum()

cuisine  \
borough       name                                                              
Bronx         19A Empire Restaurant                                   Chinese   
              2432 Peking House Inc                                   Chinese   
              2647 Double Dragon Chinese Restaurant                   Chinese   
              Best Chinese Restaurant                                 Chinese   
              Best King Wah Chinese Restaurant                        Chinese   
              Boe Sun Restaurant                                      Chinese   
              Boston Road China                                       Chinese   
              Bright Star Chinese Food                                Chinese   
              Broadway Chinese Kitchen                                Chinese   
              Brother'S Restaurant                                    Chinese   
              Century Chinese Restaurant                              Chinese   
              Chai Hwa Restaurant                                     Chinese   
              Chan'S Village                                          Chinese   
              Chef King Chinese Restaurant                            Chinese   
              Chen Ji B.B.Q Restaurant                                Chinese   
              Chen'S Foo Hai                                          Chinese   
              Chen'S Garden                                           Chinese   
              Chen'S Kitchen                                          Chinese   
              China 8 Restaurant Inc                                  Chinese   
              China City                                              Chinese   
              China Dragon City                                       Chinese   
              China Garden                                            Chinese   
              China House                                             Chinese   
              China House Restaurant                                  Chinese   
              China King                                       ChineseChinese   
              China King Ogden Restaurant                             Chinese   
              China King Restaurant                                   Chinese   
              China Kitchen                                           Chinese   
              China Mia Exotic Asian Food, Yogurt & Juice Bar         Chinese   
              China One                                               Chinese   
...                                                                       ...   
Staten Island New China Buffet                                        Chinese   
              New China Garden Restaurant                             Chinese   
              New Choi Hee Chinese                                    Chinese   
              New Everybody'S Kitchen                                 Chinese   
              New Garden Restaurant                                   Chinese   
              New Great China Restaurant                              Chinese   
              New Island                                              Chinese   
              New Island Garden                                       Chinese   
              New Phoenix                                             Chinese   
              New Phoenix Palace                                      Chinese   
              New Taste Of China                                      Chinese   
              New Win Hing Iii Chinese Restaurant                     Chinese   
              New Yung Hong                                           Chinese   
              No. 1 Chinese Kitchen                                   Chinese   
              No. 1 Kitchen                                           Chinese   
              Oriental Plaza                                          Chinese   
              Pacific Kitchen                                         Chinese   
              Panda 

In [53]:
import matplotlib.pyplot as plt
plt.close('all')

In [34]:
#plt.figure();
#df2["name"].plot()

In [35]:
#plt.pie(df2['name'], labels = {"A", "B", "C", "D", "E", "F", "G", "H", "I", "J"})

In [ ]:
import seaborn as sns
name = df2['name']
sns.countplot(y='_id', hue='name', data=df2, palette="Greens_d")

In [15]:
type(df2['name'][0])

str

In [16]:
df2.name.value_counts()

Subway                                     57
Kennedy Fried Chicken                      51
Mcdonald'S                                 42
Dunkin Donuts                              38
Dunkin' Donuts                             24
Little Caesars                             19
                                           16
Burger King                                15
Domino'S Pizza                             12
Popeyes Chicken & Biscuits                 12
Golden Krust Caribbean Bakery & Grill       9
Checkers                                    7
White Castle                                7
Carvel Ice Cream                            7
Crown Fried Chicken                         6
El Valle Restaurant                         6
Wendy'S                                     6
Jimbo'S Hamburger Palace                    5
Caridad Restaurant                          5
Applebee'S Neighborhood Grill & Bar         5
National Bakery                             4
Dunkin' Donuts, Baskin Robbins    

In [ ]:
plt.figure(figsize=(16,6))
ax = sns.countplot(x='Neighbourhood', data=dataset, order=neighborhood)
ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right", fontsize=8)
plt.title('Distribution of appointments per neighborhood', fontsize=14, fontweight='bold')
plt.show()


In [21]:
df3.borough.value_counts()

Brooklyn         763
Queens           728
Manhattan        510
Bronx            323
Staten Island     88
Missing            6
Name: borough, dtype: int64

In [23]:
df2["name"].describe()

count       2338
unique      1904
top       Subway
freq          57
Name: name, dtype: object

In [24]:

df2["borough"].describe()

count      2338
unique        1
top       Bronx
freq       2338
Name: borough, dtype: object

In [37]:
my_tab = pd.crosstab(index = df3["borough"],columns="count") 
my_tab

col_0,count
borough,
Bronx,323
Brooklyn,763
Manhattan,510
Missing,6
Queens,728
Staten Island,88


In [39]:
df2[df2['name'].isin(['Subway'])]

,_id,address,borough,cuisine,grades,name,restaurant_id
276,5c682d084f8f574fb5e0fb0f,"{'building': '3422', 'coord': {'type': 'Point'...",Bronx,Sandwiches,"[{'date': 2014-11-24 00:00:00, 'grade': 'C', '...",Subway,40862204
386,5c682d084f8f574fb5e10088,"{'building': '311', 'coord': {'type': 'Point',...",Bronx,Sandwiches,"[{'date': 2014-10-29 00:00:00, 'grade': 'Z', '...",Subway,41038490
442,5c682d084f8f574fb5e10343,"{'building': '2958', 'coord': {'type': 'Point'...",Bronx,Sandwiches,"[{'date': 2014-04-04 00:00:00, 'grade': 'A', '...",Subway,41114743
457,5c682d084f8f574fb5e103c3,"{'building': '4006', 'coord': {'type': 'Point'...",Bronx,Sandwiches,"[{'date': 2014-06-27 00:00:00, 'grade': 'A', '...",Subway,41130629
472,5c682d084f8f574fb5e1047d,"{'building': '2451', 'coord': {'type': 'Point'...",Bronx,Sandwiches,"[{'date': 2014-10-15 00:00:00, 'grade': 'B', '...",Subway,41149695
520,5c682d084f8f574fb5e106d0,"{'building': '3714', 'coord': {'type': 'Point'...",Bronx,American,"[{'date': 2014-02-04 00:00:00, 'grade': 'A', '...",Subway,41200749
568,5c682d084f8f574fb5e1087e,"{'building': '961', 'coord': {'type': 'Point',...",Bronx,Sandwiches,"[{'date': 2014-05-02 00:00:00, 'grade': 'A', '...",Subway,41235890
587,5c682d084f8f574fb5e1094f,"{'building': '38', 'coord': {'type': 'Point', ...",Bronx,Sandwiches,"[{'date': 2014-03-31 00:00:00, 'grade': 'A', '...",Subway,41247539
595,5c682d084f8f574fb5e109a8,"{'building': '5209', 'coord': {'type': 'Point'...",Bronx,Sandwiches,"[{'date': 2014-03-28 00:00:00, 'grade': 'A', '...",Subway,41253765
599,5c682d084f8f574fb5e109ea,"{'building': '968', 'coord': {'type': 'Point',...",Bronx,Sandwiches,"[{'date': 2014-03-18 00:00:00, 'grade': 'A', '...",Subway,41258837
